In [11]:
import pathlib
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
data_path = pathlib.Path('./data')
image_classes = os.listdir(data_path)
print(image_classes)

['benign', 'malignant']


In [3]:
# Load ResNet50 with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification (benign/malignant)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [4]:
# Freeze the ResNet50 layers (optional, for fine-tuning later)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
# Image preprocessing
train_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True, zoom_range=0.2, validation_split=0.2)

In [12]:
# Load train data (80%)
train_generator = train_datagen.flow_from_directory(
    data_path,  # Path to your dataset
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Use subset for training
)

# Load validation data (20%)
validation_generator = train_datagen.flow_from_directory(
    data_path,  # Same path for validation data
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Use subset for validation
)

# Model checkpoint to save the best model
checkpoint = ModelCheckpoint('resnet.h5.keras', monitor='val_accuracy', save_best_only=True, mode='max')

Found 216 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [9]:
# Train the model
history = model.fit(train_generator, 
                    validation_data=validation_generator, 
                    epochs=10, 
                    callbacks=[checkpoint])

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step - accuracy: 0.8833 - loss: 0.5095

C:\Users\Nando\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 981ms/step - accuracy: 0.8840 - loss: 0.4964 - val_accuracy: 0.8889 - val_loss: 0.3576
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 558ms/step - accuracy: 0.8894 - loss: 0.3620 - val_accuracy: 0.8889 - val_loss: 0.3990
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 566ms/step - accuracy: 0.8852 - loss: 0.4034 - val_accuracy: 0.8889 - val_loss: 0.3638
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 578ms/step - accuracy: 0.8746 - loss: 0.3939 - val_accuracy: 0.8889 - val_loss: 0.3492
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 585ms/step - accuracy: 0.8680 - loss: 0.3911 - val_accuracy: 0.8889 - val_loss: 0.3528
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 620ms/step - accuracy: 0.9036 - loss: 0.3228 - val_accuracy: 0.8889 - val_loss: 0.3493
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 600ms/step - accuracy: 0.8715 - loss: 0.3857 - val_accuracy: 0.8889 - val_loss: 0.3480
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 598ms/step - accuracy: 0.8745 - loss: 0.3767 - val_accuracy: 0.8889 - val_loss: 0.3475
Epo

In [10]:
# Save the final model
model.save('ResNet50_skin_cancer_model.h5.keras')

# Print the model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 29,885,317 (114.00 MB)

 Trainable params: 2,099,201 (8.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 4,198,404 (16.02 MB)

In [13]:
# Load the saved model
model = load_model('ResNet50_skin_cancer_model.h5.keras')

# Define the data path
data_path = pathlib.Path('./data')

# Image preprocessing for the test set
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load test data
test_generator = test_datagen.flow_from_directory(
    data_path,  # Path to your dataset (assuming test data is in the same structure)
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Shuffle should be False for evaluation
)

# Evaluate the model on the test data
evaluation = model.evaluate(test_generator)

# Print evaluation metrics
print(f"Test Loss: {evaluation[0]}")
print(f"Test Accuracy: {evaluation[1]}")

Found 270 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 397ms/step - accuracy: 0.7230 - loss: 0.6882
Test Loss: 0.3457282781600952
Test Accuracy: 0.8888888955116272
